In [1]:
import pandas as pd
import math
import numpy as np
import gc
import time

In [2]:
gc.collect()

0

In [8]:
train_common_id_2 = pd.read_parquet("data/Processed_dataset.parquet")
train_common_id_2

,common_id,event_date,event_timestamp,action_time,vacancy_id_,cookie_id,user_id,event_type,common_id_2
15,2,2022-09-05,1662374899,0.0,192850,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14
16,2,2022-09-05,1662374899,498.0,192850,0000d7508334414ca792c5ff66eb8c14,None,show_vacancy,0000d7508334414ca792c5ff66eb8c14
17,2,2022-09-05,1662375397,1.0,230581,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14
18,2,2022-09-05,1662375398,205.0,230581,0000d7508334414ca792c5ff66eb8c14,None,show_vacancy,0000d7508334414ca792c5ff66eb8c14
19,2,2022-09-05,1662375603,1.0,250327,0000d7508334414ca792c5ff66eb8c14,None,preview_click_vacancy,0000d7508334414ca792c5ff66eb8c14
...,...,...,...,...,...,...,...,...,...
12292571,367640,2022-09-26,1664169217,1.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,preview_click_vacancy,82a5e80f91b144f596496d6d831b49d3
12292572,367640,2022-09-26,1664169218,90.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,show_vacancy,82a5e80f91b144f596496d6d831b49d3
12292573,367640,2022-09-26,1664169308,210911.0,197828,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,click_contacts,82a5e80f91b144f596496d6d831b49d3
12292574,367640,2022-09-28,1664380219,132.0,233452,ffffd6d4f2134dd6aa80ff71e942b508,82a5e80f91b144f596496d6d831b49d3,click_contacts,82a5e80f91b144f596496d6d831b49d3


In [18]:
test_public_mfti_df = pd.read_parquet('data/test_public_mfti.parquet', engine='pyarrow')
test_public_mfti_df

,cookie_id,vacancy_id_
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]"
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]"
...,...,...
767,fdbcda17f22f406486837059e76c7fed,"[207851, 254989, 213344, 214180, 222146]"
768,fe6193ab26494ace9be5aae36e507618,"[115352, 230546, 225527, 120188, 109360, 23212..."
769,fe95b2826ee1452b81201ed3f4c3294d,"[240362, 114852, 253946, 251081, 127546, 244688]"
770,ff1aef256a49481698bb2e938510ff36,"[231194, 236363, 220747, 244688, 100094, 24052..."


In [22]:
# Функция для определения метрики prec@5
def precision5(cookies, recomendations):
    ratings = []
    for cookie, y_pred in zip(cookies, recomendations):
        y_test = set(test_public_mfti_df[test_public_mfti_df['cookie_id'] == cookie]['vacancy_id_'].iloc[0])
        intersected = len(set(y_test).intersection(set(y_pred)))
        rating = intersected / len(y_pred)
        ratings.append(rating)
    return sum(ratings)/len(ratings)

# ALS на common_id с фильтром на вакансии, провзаимодействованные в последний день

In [24]:
non_valid_vacs = []
for vac, group in train_mfti_df.groupby('vacancy_id_'):
    max_date = group['event_date'].max()
    if max_date <= '2022-09-30':
        non_valid_vacs.append(vac)
len(non_valid_vacs)

140242

In [27]:
train_1 = train_common_id_2[['common_id', 'vacancy_id_', 'event_type']]
train_1

,common_id,vacancy_id_,event_type
15,2,192850,preview_click_vacancy
16,2,192850,show_vacancy
17,2,230581,preview_click_vacancy
18,2,230581,show_vacancy
19,2,250327,preview_click_vacancy
...,...,...,...
12292571,367640,197828,preview_click_vacancy
12292572,367640,197828,show_vacancy
12292573,367640,197828,click_contacts
12292574,367640,233452,click_contacts


In [28]:
non_duplicated_train_1 = train_1.drop_duplicates(keep='first', ignore_index=True).sort_values(by=['common_id', 'vacancy_id_'], ignore_index=True)
non_duplicated_train_1['event_type'] = non_duplicated_train_1['event_type'].apply(lambda x: 1 if x in target_actions else 0)
non_duplicated_train_1 = non_duplicated_train_1.groupby(by=['common_id', 'vacancy_id_'], as_index=False).agg({'event_type': 'sum'})
non_duplicated_train_1

,common_id,vacancy_id_,event_type
0,1,137659,0
1,1,153975,0
2,1,174953,0
3,1,176171,0
4,1,182445,0
...,...,...,...
4723184,367641,107125,1
4723185,367641,107893,1
4723186,367641,150843,1
4723187,367641,160164,1


In [29]:
f'min rating = {non_duplicated_train_1.event_type.min()}, max rating = {non_duplicated_train_1.event_type.max()}'

'min rating = 0, max rating = 6'

In [30]:
# Приводим common_id и vacancy_id к категориальному типу для дальнейшего использования матрице взаимодействий
non_duplicated_train_1.event_type = non_duplicated_train_1.event_type.astype('uint8')
non_duplicated_train_1.common_id = non_duplicated_train_1.common_id.astype('category')
non_duplicated_train_1.vacancy_id_ = non_duplicated_train_1.vacancy_id_.astype('category')
non_duplicated_train_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4723189 entries, 0 to 4723188
Data columns (total 3 columns):
 #   Column       Dtype   
---  ------       -----   
 0   common_id    category
 1   vacancy_id_  category
 2   event_type   uint8   
dtypes: category(2), uint8(1)
memory usage: 56.7 MB


In [31]:
from scipy.sparse import csr_matrix

# Строим матрицы взаимодействий
# Vacancy_id будет представлен кодами категорий, начинающимися с 0 (то есть 100001 номер вакансии будет соответствовать
# 0 индексу кода категории, 100002 - 1 индексу и т.д.)
item_user_1 = csr_matrix((non_duplicated_train_1['event_type'], 
                           (non_duplicated_train_1['vacancy_id_'].cat.codes, 
                            non_duplicated_train_1['common_id'].cat.codes)))

user_item_1 = csr_matrix((non_duplicated_train_1['event_type'], 
                           (non_duplicated_train_1['common_id'].cat.codes, 
                            non_duplicated_train_1['vacancy_id_'].cat.codes)))

id_cat_codes_1 = dict(zip(non_duplicated_train_1['common_id'], non_duplicated_train_1['common_id'].cat.codes))
vac_cat_codes_1 = dict(zip(non_duplicated_train_1['vacancy_id_'], non_duplicated_train_1['vacancy_id_'].cat.codes))
cat_vac_codes_1 = dict(zip(non_duplicated_train_1['vacancy_id_'].cat.codes, non_duplicated_train_1['vacancy_id_']))

In [32]:
from implicit.als import AlternatingLeastSquares

model_1 = AlternatingLeastSquares(factors=100, regularization=0.05, alpha=3.0, calculate_training_loss=True, random_state=42)
model_1.fit(user_item_1)

  0%|          | 0/15 [00:00<?, ?it/s]

In [33]:
import time
start = time.time()
cookies = test_public_mfti_df.cookie_id.to_list()
non_valid_vacs_recommend = list(map(lambda x: x - 100001, non_valid_vacs))
recomendations = []
for cookie in cookies:
    common_ids = common_from_cookie(cookie)
    recs = []
    for com_id, n in common_ids.items():
        if n == 0:
            continue
        actioned_vacs = train_common_id_2[train_common_id_2['common_id'] == com_id].vacancy_id_.unique()
        actioned_vacs = actioned_vacs - 100001
        filter_items = np.concatenate((actioned_vacs, non_valid_vacs_recommend))
        ids, scores = model_1.recommend(id_cat_codes_1[com_id], user_item_1[id_cat_codes_1[com_id]], N=n, filter_items=filter_items, filter_already_liked_items = True)
        recs.extend([ident+100001 for ident in ids])
    recomendations.append(recs)
end = time.time()
print(f'Потреченное время: {end-start}')

Потреченное время: 590.8551938533783


In [34]:
precision5(cookies, recomendations)

0.046891191709844515